In [1]:
import glob
import matplotlib.pyplot as plt
import nana
from astropy.table import Table
from astropy.io import ascii
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
star_files = glob.glob("*.dat")
#star_files.remove('kplr010536147.dat')
#print(len(star_files))

In [3]:
print(star_files)

['kplr003865742.dat', 'kplr004930889.dat', 'kplr010526294.dat', 'kplr009715425.dat', 'kplr009020774.dat', 'kplr006462033.dat', 'kplr006352430.dat', 'kplr010536147.dat', 'kplr008459899.dat', 'kplr003459297.dat', 'kplr008255796.dat', 'kplr008714886.dat', 'kplr003240411.dat', 'kplr006780397.dat', 'kplr008057661.dat', 'kplr012258330.dat', 'kplr011360704.dat', 'kplr011971405.dat', 'kplr004936089.dat', 'kplr008766405.dat', 'kplr005941844.dat', 'kplr007630417.dat', 'kplr007760680.dat', 'kplr004939281.dat', 'kplr005309849.dat', 'kplr008381949.dat']


In [6]:
num = 0

for star in star_files:
    data = pd.read_csv(star)
    data = pd.read_csv(star, sep=r'\s+', header=None, names=['time', 'flux'])
    star = star[:13]

    print("working on ", star)
    
    #pipeline 
    lc, delta_f, sampling_time = nana.star(star)
    
    
    f_max = (3 / (2*sampling_time))
    f_min = delta_f/3
    t_fit, flux_fit, weight_fit = nana.mask_vals(lc)
    freq_full, power_full = nana.pg_full(f_min, f_max, lc)
    freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
    indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
    refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
    fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
    regions, final_freqs, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit, flux_fit, weight_fit)
    a_early, a_late, b_early, b_late = nana.check_coherence(t_fit, flux_fit, weight_fit, final_freqs)
    rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit)
    delta_chi2s = nana.null_chi_squared(t_fit, flux_fit, weight_fit) - chi2s
    sharpnesses = nana.sharpness(second_derivatives, refined_power)
    
    
    data = Table()


    data['modes'] = final_freqs
    data['regions'] = regions
    data['corresponding_freq_in_region_A'] = refined_freq
    data['sharpness'] = sharpnesses
    data['delta_chi_squares'] = delta_chi2s
    data['rate_of_change_of_phase'] = rate_of_phase
    data['rate_of_change_of_amplitude'] = rate_of_amp
    
    ascii.write(
        data,
        star + '.csv',
        overwrite=True,
        format="csv",
        formats={
            "modes": "{:.10f}",
            "corresponding_freq_in_region_A": "{:.5f}",
            "sharpness": "{:.7e}",
            "delta_chi_squares": "{:.7e}",
            "rate_of_change_of_phase": "{:.7e}",
            "rate_of_change_of_amplitude": "{:.7e}"
        }
    )
    
    num = num + 1
    print(num)
    
    

working on  kplr003865742
1
working on  kplr004930889
2
working on  kplr010526294
times not in order


AssertionError: 

In [5]:
lc, delta_f, sampling_time = nana.star('kplr010536147')

times not in order


AssertionError: 

In [ ]:
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit, flux_fit, weight_fit = nana.mask_vals(lc)

In [ ]:
print(sampling_time * 86400)
print(len(lc))

In [ ]:
plt.plot(lc.time.value)

In [ ]:
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)

In [ ]:
plt.scatter(freq_mini[indices], refined_power, color = 'r', marker = 'o')
plt.plot(freq_mini, power_mini, 'k-')
plt.title("star that won't work")

plt.semilogy()

In [ ]:
freq_mini[indices]

In [ ]:
plt.plot(freq_full, power_full, 'k.')

In [ ]:
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit, flux_fit, weight_fit)

In [ ]:
a_early, a_late, b_early, b_late = nana.check_coherence(t_fit, flux_fit, weight_fit, final_freqs)

In [ ]:
rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit)
delta_chi2s = nana.null_chi_squared(t_fit, flux_fit, weight_fit) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
def region_and_freq(indices, folding_freq, f_min, unrefined_freq, unrefined_power, t_fit, flux_fit, weight_fit):
    regions, best_freqs, best_chi2s = [], [], []

    for i in range(len(indices)):
        fas, __, __ = refine_peaks(unrefined_freq, unrefined_power, indices)
        A, B, C = fas, folding_freq - fas, folding_freq + fas

        fine_freqsA = np.arange(A[i] - 5 * f_min, A[i] + 5 * f_min, 0.2 * f_min)
        chi2_fineA = np.array([integral_chi_squared(2. * np.pi * f, t_fit, flux_fit, weight_fit, lc_exptime) for f in fine_freqsA])
        best_freqA, best_chi2A = find_min_and_refine(fine_freqsA, chi2_fineA)

        fine_freqsB = np.arange(B[i] - 5 * f_min, B[i] + 5 * f_min, 0.2 * f_min)
        chi2_fineB = np.array([integral_chi_squared(2. * np.pi * f, t_fit, flux_fit, weight_fit, lc_exptime) for f in fine_freqsB])
        best_freqB, best_chi2B = find_min_and_refine(fine_freqsB, chi2_fineB)

        fine_freqsC = np.arange(C[i] - 5 * f_min, C[i] + 5 * f_min, 0.2 * f_min)
        chi2_fineC = np.array([integral_chi_squared(2. * np.pi * f, t_fit, flux_fit, weight_fit, lc_exptime) for f in fine_freqsC])
        best_freqC, best_chi2C = find_min_and_refine(fine_freqsC, chi2_fineC)

        if best_chi2A <= best_chi2B and best_chi2A <= best_chi2C:
            regions.append("A")
            best_freqs.append(best_freqA)
            best_chi2s.append(best_chi2A)
        elif best_chi2B < best_chi2A and best_chi2B < best_chi2C:
            regions.append("B")
            best_freqs.append(best_freqB)
            best_chi2s.append(best_chi2B)
        elif best_chi2C < best_chi2A and best_chi2C < best_chi2B:
            regions.append("C")
            best_freqs.append(best_freqC)
            best_chi2s.append(best_chi2C)

    return regions, best_freqs, best_chi2s


In [ ]:
def find_min_and_refine(xs,ys):

    indxs, properties = find_peaks(-ys)
    min_index =  indxs[np.argsort(ys[indxs])[:1]]
    refined_x, refined_y, second_derivative = refine_peaks(xs, ys, min_index)

    return refined_x[0], refined_y[0]


In [ ]:
def refine_peaks(xs, ys, indices):
    
    foo = lambda i: refine_peak(xs, ys, i)
    xs_refined, ys_refined, second_derivatives = zip(*list(map(foo, indices)))
    return np.array(xs_refined), np.array(ys_refined), np.array(second_derivatives)
